In [10]:
pip install transformers[torch]

^C
Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install tf-keras


Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install datasets scikit-learn

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
    --------------------------------------- 0.3/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 1.4 MB/s eta 0:00:08
   --- ------------------------------------ 1.0/11.1 MB 1.6 MB/s eta 0:00:07
   ---- ----------------------------------- 1.3/11.1 MB 1.5 MB/s eta 0:00:07
   ------ --------------------------------- 1.8/11.1 MB 1.7 MB/s eta 0:00:06
   ------- -------------------------------- 2.1/11.1 MB 1.7 MB/s eta 0:00:06
   --------- ------------------------------ 2.6/11.1 MB 1.8 MB/s eta 0:00:05
   ----------- ---------------------------- 3.1/11.1 MB 1.8 MB/s eta 0:00:05
   ------------- -------------------------- 3.7/11.1 MB 1.9 MB/s eta 0:00:04
   --------------- ------------------------ 4.2/11.1 MB 2.0 MB/s eta 0:00:04
   ---------------- --------------

In [2]:
import torch

In [3]:
from transformers import (
    GPT2Tokenizer, 
    GPT2LMHeadModel, 
    TrainingArguments, 
    Trainer, 
    DataCollatorForLanguageModeling
)
from datasets import load_dataset
from sklearn.model_selection import train_test_split

c:\Users\agboo\.conda\envs\openvino\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
fd = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en") # Load the dataset
ds = fd['train'].select(range(50)) # Adjusted to select the first 300 samples

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
import os
model_save_path = './medical_gpt2_finetuned'
os.makedirs(model_save_path, exist_ok=True)

In [9]:
# Preprocess the dataset
def preprocess_data(examples):
    # Combine input and output for fine-tuning
    texts = [f"Input: {inp}\nOutput: {out}" for inp, out in zip(examples['Question'], examples['Response'])]
    return {"text": texts}

In [13]:
# Apply preprocessing
processed_dataset = ds.map(preprocess_data, batched=True, remove_columns=ds.column_names)


In [11]:
ds.column_names

['Question', 'Complex_CoT', 'Response']

In [14]:
from datasets import Dataset

train_data = processed_dataset.to_list()
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)


train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

print(f"Training size: {len(train_dataset)}, Validation size: {len(val_dataset)}")

Training size: 40, Validation size: 10


In [15]:
# Load tokenizer and model
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)

In [16]:
# Tokenize datasets
def tokenize_function(examples):
    return tokenizer(
        examples['text'], 
        truncation=True, 
        padding='max_length', 
        max_length=512
    )

train_tokenized = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
val_tokenized = val_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

Map: 100%|██████████| 10/10 [00:00<00:00, 42.95 examples/s]


In [17]:
# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [18]:
# Training arguments
training_args = TrainingArguments(
    output_dir=model_save_path,
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    fp16=torch.cuda.is_available(),
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    data_collator=data_collator,
)



c:\Users\agboo\.conda\envs\openvino\lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
# Train with early stopping (implicit via load_best_model_at_end)
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)
print("Training completed and model saved!")

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
50,2.410800,2.603300


Training completed and model saved!


In [20]:
# Generate text
from transformers import pipeline

generator = pipeline('text-generation', model=model_save_path, tokenizer=model_save_path)

# Test prompts
test_prompts = [
    "Question: What are the symptoms of",
    "Question: Explain the treatment for",
    "Question: Describe the diagnosis of"
]

for prompt in test_prompts:
    output = generator(
        prompt,
        max_length=200,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True,
        truncation=True
    )
    print(f"\nPrompt: {prompt}")
    print(f"Generated Text: {output[0]['generated_text']}")

Device set to use cpu



Prompt: Question: What are the symptoms of
Generated Text: Question: What are the symptoms of posttraumatic stress disorder?
Output: This is a diagnosis that can be made at any age. In this case, PTSD requires immediate attention and treatment by an experienced psychiatric professional to resolve problems associated with trauma including depression, anxiety, fear, panic attacks or paranoia during difficult times such as emergency situations where mental health conditions have become too overwhelming for coping appropriately.[1] While there may not be sufficient physical evidence in most cases indicating specific underlying psychological issues related specifically back pain (as reported previously), it would certainly help if someone was experiencing some form from what they've described above about their emotional state after witnessing these events on television—such as hearing voices yelling "Help!" versus standing up straight-up without looking down; seeing other people's faces wh